## newnetwork2.ipynb
An improved version of network.py, implementing the stochastic gradient descent learning algorithm
for a feedforward neural network. Improvements include the addition of the cross-entropy cost function,
regularization, and better initialization of network weights.

### Libraries

In [1]:
import random
import sys
import numpy as np

### Define the quadratic and cross-entropy cost functions

In [2]:
class QuadraticCost(object):

    @staticmethod
    def fn(a, y):
        # Return the cost associated with an output ``a`` and desired output ``y``.

        return 0.5*np.linalg.norm(a-y)**2

    @staticmethod
    def delta(z, a, y):
        # Return the error delta from the output layer.

        return (a-y) * sigmoid_prime(z)

In [3]:
class CrossEntropyCost(object):

    @staticmethod
    def fn(a, y):
        # Return the cost associated with an output ``a`` and desired output ``y``. 
        # Note that np.nan_to_num is used to ensure numerical stability. In particular, 
        # if both ``a`` and ``y`` have a 1.0 in the same slot, then the expression (1-y)*np.log(1-a)
        # returns nan.  The np.nan_to_num ensures that that is converted to the correct value (0.0).

        epsilon = 1e-10
        a = np.clip(a, epsilon, 1 - epsilon)
        return np.sum(-y * np.log(a) - (1 - y) * np.log(1 - a))


    @staticmethod
    def delta(z, a, y):
        # Return the error delta from the output layer. Note that the parameter ``z`` is not used by the method.
        # It is included in the method's parameters in order to make the interface consistent with the delta method
        # for other cost classes.

        return (a-y)

In [4]:
#### Main Network class
class Network(object):

    def __init__(self, sizes, cost=CrossEntropyCost):
        # The list ``sizes`` contains the number of neurons in the respective layers of the network.
        # For example, if the list was [2, 3, 1] then it would be a three-layer network, with the first layer
        # containing 2 neurons, the second layer 3 neurons, and the third layer 1 neuron. The biases and weights
        # for the network are initialized randomly, using ``self.default_weight_initializer``

        self.num_layers = len(sizes)
        self.sizes = sizes
        self.default_weight_initializer()
        self.cost=cost

    def default_weight_initializer(self):
        # Initialize each weight using a Gaussian distribution with mean 0 and standard deviation 1 over the square root
        # of the number of weights connecting to the same neuron. Initialize the biases using a Gaussian distribution
        # with mean 0 and standard deviation 1.

        # Note that the first layer is assumed to be an input layer, and by convention we won't set any biases for 
        # those neurons, since biases are only ever used in computing the outputs from later layers.

        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)/np.sqrt(x) for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def large_weight_initializer(self):
        # Initialize the weights using a Gaussian distribution with mean 0 and standard deviation 1.
        # Initialize the biases using a Gaussian distribution with mean 0 and standard deviation 1.


        # This weight and bias initializer uses the same approach as in Chapter 1, and is included for purposes of comparison.
        # It will usually be better to use the default weight initializer instead.

        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x) / np.sqrt(x) for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a): 
        for b, w in zip(self.biases[:-1], self.weights[:-1]):
            a = tanh(np.dot(w, a) + b)
        a = sigmoid(np.dot(self.weights[-1], a) + self.biases[-1])
        return a


    def SGD(self, training_data, epochs, mini_batch_size, eta,
            lmbda = 0.0,
            evaluation_data=None,
            monitor_evaluation_cost=False,
            monitor_evaluation_accuracy=False,
            monitor_training_cost=False,
            monitor_training_accuracy=False):

        # Train the neural network using mini-batch stochastic gradient descent. The ``training_data`` is a list of
        # tuples ``(x, y)`` representing the training inputs and the desired outputs. The other non-optional parameters
        # are self-explanatory, as is the regularization parameter ``lmbda``.  The method also accepts ``evaluation_data``,
        # usually either the validation or test data. We can monitor the cost and accuracy on either the evaluation data
        # or the training data, by setting the appropriate flags. The method returns a tuple containing four lists: 
        # the (per-epoch) costs on the evaluation data, the accuracies on the evaluation data,
        # the costs on the training data, and the accuracies on the training data. 
        # All values are evaluated at the end of each training epoch. So, for example, if we train for 30 epochs,
        # then the first element of the tuple will be a 30-element list containing the cost on the evaluation data at the end
        # of each epoch. Note that the listsare empty if the corresponding flag is not set.

        if evaluation_data:
            n_data = len(evaluation_data)
        n = len(training_data)
        evaluation_cost, evaluation_accuracy = [], []
        training_cost, training_accuracy = [], []
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(
                    mini_batch, eta, lmbda, len(training_data))
            print("Epoch %s training complete" % j)
            if monitor_training_cost:
                cost = self.total_cost(training_data, lmbda)
                training_cost.append(cost)
                print("Cost on training data: {}".format(cost))
            if monitor_training_accuracy:
                accuracy = self.accuracy(training_data, convert=True)
                training_accuracy.append(accuracy)
                print("Accuracy on training data: {} / {}".format(
                    accuracy, n))
            if monitor_evaluation_cost:
                cost = self.total_cost(evaluation_data, lmbda, convert=True)
                evaluation_cost.append(cost)
                print("Cost on evaluation data: {}".format(cost))
            if monitor_evaluation_accuracy:
                accuracy = self.accuracy(evaluation_data)
                evaluation_accuracy.append(accuracy)
                print("Accuracy on evaluation data: {} / {}".format(
                    self.accuracy(evaluation_data), n_data))

        return evaluation_cost, evaluation_accuracy, training_cost, training_accuracy

    def update_mini_batch(self, mini_batch, eta, lmbda, n):
        # Update the network's weights and biases by applying gradient descent using backpropagation to a single mini batch.
        # The ``mini_batch`` is a list of tuples ``(x, y)``, ``eta`` is the learning rate, ``lmbda`` is the regularization
        # parameter, and ``n`` is the total size of the training data set.

        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [(1-eta*(lmbda/n))*w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        # Return a tuple ``(nabla_b, nabla_w)`` representing the gradient for the cost function C_x.
        # ``nabla_b`` and ``nabla_w`` are layer-by-layer lists of numpy arrays, similar to ``self.biases`` and ``self.weights``.

        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = (self.cost).delta(zs[-1], activations[-1], y)
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        # Note that the variable l in the loop below is used a little differently to the notation in Chapter 2 of the book.
        # Here, l = 1 means the last layer of neurons, l = 2 is the second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact that Python can use negative indices in lists.

        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = tanh_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
            
        return (nabla_b, nabla_w)

    def accuracy(self, data, convert=False):
        # Return the number of inputs in ``data`` for which the neural network outputs the correct result. The neural network's
        # output is assumed to be the index of whichever neuron in the final layer has the highest activation.

        # The flag ``convert`` should be set to False if the data set is validation or test data (the usual case), and to True
        # if the data set is the training data. The need for this flag arises due to differences in the way the results ``y`` are
        # represented in the different data sets.  In particular, it flags whether we need to convert between the different
        # representations. It may seem strange to use different representations for the different data sets. Why not use the
        # same representation for all three data sets?  It's done for efficiency reasons -- the program usually evaluates the cost
        # on the training data and the accuracy on other data sets. These are different types of computations, and using different
        # representations speeds things up. More details on the representations can be found in mnist_loader.load_data_wrapper.

        if convert:
            results = [(np.argmax(self.feedforward(x)), np.argmax(y)) for (x, y) in data]
        else:
            results = [(np.argmax(self.feedforward(x)), y) for (x, y) in data]
        return sum(int(x == y) for (x, y) in results)

    def total_cost(self, data, lmbda, convert=False):
        # Return the total cost for the data set ``data``. The flag ``convert`` should be set to False if the data set is the
        # training data (the usual case), and to True if the data set is the validation or test data.

        cost = 0.0
        for x, y in data:
            a = self.feedforward(x)
            if convert: y = vectorized_result(y)
            cost += self.cost.fn(a, y)/len(data)
        cost += 0.5*(lmbda/len(data))*sum(np.linalg.norm(w)**2 for w in self.weights)
        return cost

### Miscellaneous functions

In [5]:
def vectorized_result(j):
    # Return a 10-dimensional unit vector with a 1.0 in the j'th position and zeroes elsewhere. This is used to convert
    # a digit (0...9) into a corresponding desired output from the neural network.

    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [6]:
def sigmoid(z):
    # The sigmoid function.

    return 1.0/(1.0+np.exp(-z))

In [7]:
def sigmoid_prime(z):
    # Derivative of the sigmoid function.

    return sigmoid(z)*(1-sigmoid(z))

In [8]:
#New functions
def tanh(z):
    return np.tanh(z)

def tanh_prime(z):
    return 1.0 - np.tanh(z)**2


## mnist_loader

A library to load the MNIST image data. For details of the data structures that are returned, see the doc strings for
``load_data`` and ``load_data_wrapper``. In practice, ``load_data_wrapper`` is the function called by neural network code.

### Libraries

In [9]:
import pickle
import gzip

In [10]:
def load_data():
    # Return the MNIST data as a tuple containing the training data, the validation data, and the test data.
    # The ``training_data`` is returned as a tuple with two entries. The first entry contains the actual training images.
    # This is a numpy ndarray with 50,000 entries. Each entry is, in turn, a numpy ndarray with 784 values, representing
    # the 28 * 28 = 784 pixels in a single MNIST image. The second entry in the ``training_data`` tuple is a numpy ndarray
    # containing 50,000 entries. Those entries are just the digit values (0...9) for the corresponding images contained 
    # in the first entry of the tuple.

    # The ``validation_data`` and ``test_data`` are similar, except each contains only 10,000 images.

    f = gzip.open('mnist.pkl.gz', 'rb')
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    training_data, validation_data, test_data = u.load()
    f.close()
    return (training_data, validation_data, test_data)

In [11]:
def load_data_wrapper():
    # Return a tuple containing ``(training_data, validation_data, test_data)``. Based on ``load_data``,
    # but the format is more convenient for use in our implementation of neural networks.

    # In particular, ``training_data`` is a list containing 50,000 2-tuples ``(x, y)``.  ``x`` is a 784-dimensional 
    # numpy.ndarray containing the input image.  ``y`` is a 10-dimensional numpy.ndarray representing the unit vector
    # corresponding to the correct digit for ``x``.

    # ``validation_data`` and ``test_data`` are lists containing 10,000 2-tuples ``(x, y)``.  In each case, 
    # ``x`` is a 784-dimensional numpy.ndarry containing the input image, and ``y`` is the corresponding classification,
    # i.e., the digit values (integers) corresponding to ``x``.
    
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = list(zip(training_inputs, training_results))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    return (training_data, validation_data, test_data)

In [12]:
def run_experiments():

    # Run the experiments described in chapter 1 of the book.

    training_data, validation_data, test_data = load_data_wrapper()
    net = Network([784, 30, 10], cost = CrossEntropyCost)
    net.SGD(training_data, 30, 10, 0.5, lmbda = 5.0, evaluation_data = validation_data, monitor_evaluation_accuracy = True, \
            monitor_evaluation_cost = True, monitor_training_accuracy = True, monitor_training_cost = True)

It is time to call the function run_experiments() to execute all the commands

In [13]:
run_experiments()

Epoch 0 training complete
Cost on training data: 23.797132562560694
Accuracy on training data: 38476 / 50000
Cost on evaluation data: 23.99379738585636
Accuracy on evaluation data: 7881 / 10000
Epoch 1 training complete
Cost on training data: 20.29147600084242
Accuracy on training data: 42575 / 50000
Cost on evaluation data: 20.885050415248667
Accuracy on evaluation data: 8646 / 10000
Epoch 2 training complete
Cost on training data: 24.082586120405832
Accuracy on training data: 40048 / 50000
Cost on evaluation data: 24.676984844195847
Accuracy on evaluation data: 8166 / 10000
Epoch 3 training complete
Cost on training data: 23.016488802578824
Accuracy on training data: 37949 / 50000
Cost on evaluation data: 23.618293190985398
Accuracy on evaluation data: 7782 / 10000
Epoch 4 training complete
Cost on training data: 21.993851870491184
Accuracy on training data: 39653 / 50000
Cost on evaluation data: 22.871247813889028
Accuracy on evaluation data: 8080 / 10000
Epoch 5 training complete
C